## Spam mail分類を行う

In [31]:
%matplotlib inline

import libs.load_spam_data



[nltk_data] Downloading package punkt to /Users/nisida/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nisida/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


{'name': 1, 'price':2} ->  all_token内の全ての単語にユニークなID（数字)をふる（Pythonの辞書で作成する）. -> コーパスと呼ぶ。

気になるpoint

token1 = [1,3, 6, 7] , token2 = [10, 13, 14, 15 16, 17]


In [ ]:
import nltk 

print(nltk.__version__)